## 引言 

本课程将涵盖： 
- 函数调用的概念及其用例 
- 如何使用 OpenAI 创建函数调用 
- 如何将函数调用集成到应用程序中 

## 学习目标 

完成本课程后，您将了解并掌握： 

-  使用函数调用的目的 
- 使用 OpenAI 服务设置函数调用 
- 为您的应用程序用例设计有效的函数调用

## 理解函数调用 

在本课程中，我们希望为我们的教育初创公司构建一个功能，允许用户使用聊天机器人查找技术课程。我们将推荐适合他们技能水平、当前角色和感兴趣的技术的课程。 

为了完成此任务，我们将结合使用： 
 - `OpenAI` 创建用户聊天体验
 - `Microsoft Learn Catalog API` 根据用户请求帮助用户查找课程 
 - `Function Calling` 获取用户查询并将其发送到函数以进行 API 请求。 

首先，让我们看看为什么我们首先要使用函数调用： 

print("下一次请求中的消息：")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # 从 GPT 获取新的响应，以便它可以看到函数响应


print(second_response.choices[0].message)

### 为什么使用函数调用 

如果您完成了本课程中的任何其他课程，您可能已经了解了使用大型语言模型 (LLM) 的强大之处。希望您也能看到它们的一些局限性。 

函数调用是 OpenAI 服务的一项功能，旨在解决以下挑战：

响应格式不一致：
- 在函数调用之前，大型语言模型的响应是非结构化且不一致的。开发人员必须编写复杂的验证代码来处理输出中的每种变化。

与外部数据的集成有限：
- 在此功能之前，很难将应用程序其他部分的数据整合到聊天上下文中。

通过标准化响应格式并实现与外部数据的无缝集成，函数调用简化了开发并减少了对额外验证逻辑的需求。

用户无法获得诸如“斯德哥尔摩当前天气如何？”之类的答案。这是因为模型受限于训练数据的时间。 

让我们看下面的示例来说明这个问题： 

假设我们想创建一个学生数据库，以便我们可以向他们推荐合适的课程。下面我们有两个学生描述，它们包含的数据非常相似。

In [12]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."


我们希望将这些数据发送到 LLM 进行解析。这以后可以在我们的应用程序中用于发送到 API 或存储在数据库中。 

让我们创建两个相同的提示，指导 LLM 提取我们感兴趣的信息：

我们希望将这些数据发送到 LLM，以解析对我们的产品重要的部分。因此，我们可以创建两个相同的提示来指导 LLM：

In [13]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object. 
Note: I can parse JSON objects, so please do not add any additional text or format, and do not use any markdown format(```json content```).:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''

print(prompt1)


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object. 
Note: I can parse JSON objects, so please do not add any additional text or format, and do not use any markdown format(```json content```).:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''



Please extract the following information from the given text and return it as a JSON object. 
Note: I can parse JSON objects, so please do not add any additional text or format, and do not use any markdown format(```json content```).:

name
major
school
grades
club

This is the body of text to extract the information from:
Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating.



创建这两个提示后，我们将使用 `openai.ChatCompletion` 将它们发送到 LLM。我们将提示存储在 `messages` 变量中，并将角色分配给 `user`。这是为了模拟用户向聊天机器人发送消息。

In [14]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="deepseek-chat"

现在我们可以将这两个请求发送到 LLM 并检查收到的响应。

In [15]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)

openai_response1.choices[0].message.content 

'{\n  "name": "Emily Johnson",\n  "major": "computer science",\n  "school": "Duke University",\n  "grades": "3.7 GPA",\n  "club": ["Chess Club", "Debate Team"]\n}'

In [16]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

'{\n  "name": "Michael Lee",\n  "major": "computer science",\n  "school": "Stanford University",\n  "grades": "3.8 GPA",\n  "club": "Robotics Club"\n}'

In [17]:
# 将响应加载为 JSON 对象
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

{'name': 'Emily Johnson',
 'major': 'computer science',
 'school': 'Duke University',
 'grades': '3.7 GPA',
 'club': ['Chess Club', 'Debate Team']}

In [18]:
# 将响应加载为 JSON 对象
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

{'name': 'Michael Lee',
 'major': 'computer science',
 'school': 'Stanford University',
 'grades': '3.8 GPA',
 'club': 'Robotics Club'}


即使提示相同且描述相似，我们也可以获得不同格式的 `Grades` 属性。 

如果您多次运行上面的单元格，格式可能是 `3.7` 或 `3.7 GPA`。 

这是因为 LLM 接收书面提示形式的非结构化数据，并返回非结构化数据。我们需要一种结构化格式，以便在存储或使用此数据时知道会发生什么

通过使用函数调用，我们可以确保接收到结构化数据。使用函数调用时，LLM 实际上不会调用或运行任何函数。相反，我们为 LLM 创建一个结构，以便它遵循该结构进行响应。然后，我们使用这些结构化响应来知道在应用程序中运行哪个函数。  
 

![Function Calling Flow Diagram](../images/Function-Flow.png?WT.mc_id=academic-105485-koreyst)

然后我们可以将函数返回的内容发送回 LLM。LLM 将使用自然语言响应以回答用户的查询。

### 函数调用的用例 

**调用外部工具** 
聊天机器人非常擅长回答用户的问题。通过使用函数调用，聊天机器人可以使用用户的消息来完成某些任务。例如，学生可以要求聊天机器人“发送电子邮件给我的老师，说我需要更多关于这个主题的帮助”。这可以调用 `send_email(to: string, body: string)` 函数。


**创建 API 或数据库查询**
用户可以使用自然语言查找信息，然后将其转换为格式化的查询或 API 请求。一个例子是老师请求“完成上次作业的学生是谁”，这可以调用名为 `get_completed(student_name: string, assignment: int, current_status: string)` 的函数。


**创建结构化数据**
用户可以获取一段文本或 CSV，并使用 LLM 从中提取重要信息。例如，学生可以将关于和平协议的维基百科文章转换为创建 AI 闪卡。这可以通过调用名为 `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` 的函数来完成。

## 2. 创建您的第一个函数调用 

创建函数调用的过程包括 3 个主要步骤： 
1. 使用您的函数列表和用户消息调用 Chat Completions API 
2. 读取模型的响应以执行操作，即执行函数或 API 调用 
3. 使用函数返回的响应再次调用 Chat Completions API，以使用该信息创建对用户的响应。

![Flow of a Function Call](../images/LLM-Flow.png?WT.mc_id=academic-105485-koreyst)

### 函数调用的元素 

#### 用户输入 

第一步是创建用户消息。这可以通过获取文本输入的值动态分配，或者您可以在此处分配一个值。如果这是您第一次使用 Chat Completions API，我们需要定义消息的 `role` 和 `content`。 

`role` 可以是 `system`（创建规则）、`assistant`（模型）或 `user`（最终用户）。对于函数调用，我们将此分配为 `user` 并提供一个示例问题。

In [19]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure. Use tools(function-calling)"} ]

### 创建函数。 

接下来我们将定义一个函数及其参数。这里我们只使用一个名为 `search_courses` 的函数，但您可以创建多个函数。

**重要**：函数包含在发送给 LLM 的系统消息中，并将计入您可用的令牌数量。

In [23]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

tools = [
  {
    "type": "function",
    "function": functions[0],
  },
]

**定义** 

函数定义结构具有多个级别，每个级别都有自己的属性。以下是嵌套结构的细分：

**顶级函数属性：**

`name` - 我们想要调用的函数的名称。

`description` - 这是函数工作方式的描述。在这里，具体和清晰很重要。

`parameters` - 您希望模型在其响应中生成的值和格式列表

**参数对象属性：**

`type` - 参数对象的数据类型（通常为“object”）

`properties` - 模型将用于其响应的特定值列表

**单个参数属性：**

`name` - 由属性键隐式定义（例如，“role”、“product”、“level”）

`type` - 此特定参数的数据类型（例如，“string”、“number”、“boolean”）

`description` - 特定参数的描述

**可选属性：**

`required` - 列出完成函数调用所需的参数的数组

### 进行函数调用 
定义函数后，我们现在需要将其包含在对 Chat Completion API 的调用中。我们通过向请求添加 `functions` 来实现这一点。在这种情况下，`functions=functions`。

还有一个选项可以将 `function_call` 设置为 `auto`。这意味着我们将让 LLM 根据用户消息决定调用哪个函数，而不是我们自己指定。

In [24]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        tools=tools, 
                                        tool_choice="auto") 

print(response.choices[0].message)

ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_acb35831-f532-4184-8c75-99f9036fed64', function=Function(arguments='{"role":"student","product":"Azure","level":"beginner"}', name='search_courses'), type='function', index=0)])


现在让我们看看响应并了解其格式： 

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

您可以看到函数名称被调用，并且 LLM 能够从用户消息中找到适合函数参数的数据。

## 3. 将函数调用集成到应用程序中 


在测试了 LLM 的格式化响应后，现在我们可以将其集成到应用程序中。 

### 管理流程 

为了将其集成到我们的应用程序中，让我们执行以下步骤： 

首先，让我们调用 OpenAI 服务并将消息存储在名为 `response_message` 的变量中。

In [25]:
response_message = response.choices[0].message

现在我们将定义调用 Microsoft Learn API 以获取课程列表的函数：

In [26]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



作为最佳实践，我们将查看模型是否要调用函数。之后，我们将创建一个可用函数并将其与正在调用的函数匹配。
然后，我们将获取函数的参数，并将其映射到 LLM 的参数。

最后，我们将附加函数调用消息和 `search_courses` 消息返回的值。这为 LLM 提供了使用自然语言响应用户所需的所有信息。

In [31]:
# 检查模型是否要调用函数
if response_message.tool_calls[0].function.name:
    print("推荐的函数调用：")
    print(response_message.tool_calls[0].function.name)
    print()

    # 调用函数。
    function_name = response_message.tool_calls[0].function.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.tool_calls[0].function.arguments)
    function_response = function_to_call(**function_args)

    print("函数调用的输出：")
    print(function_response)
    print(type(function_response))


    # 将助手响应和函数响应添加到消息中
    messages.append( # 将助手响应添加到消息中
        response_message
    )
    messages.append( # 将函数响应添加到消息中
        {
            "role": "tool",
            "tool_call_id": response_message.tool_calls[0].id,
            "content":function_response,
        }
    )



推荐的函数调用：
search_courses

函数调用的输出：
[{'title': 'Describe concepts of cryptography', 'url': 'https://learn.microsoft.com/en-us/training/modules/describe-concepts-of-cryptography/?WT.mc_id=api_CatalogApi'}, {'title': 'Introduction to audio classification with TensorFlow', 'url': 'https://learn.microsoft.com/en-us/training/modules/intro-audio-classification-tensorflow/?WT.mc_id=api_CatalogApi'}, {'title': 'Design a Performant Data Model in Azure SQL Database with Azure Data Studio', 'url': 'https://learn.microsoft.com/en-us/training/modules/design-a-data-model-with-ads/?WT.mc_id=api_CatalogApi'}, {'title': 'Fundamentals of Azure AI Speech', 'url': 'https://learn.microsoft.com/en-us/training/modules/recognize-synthesize-speech/?WT.mc_id=api_CatalogApi'}, {'title': 'Host a web application with Azure App Service', 'url': 'https://learn.microsoft.com/en-us/training/modules/host-a-web-app-with-azure-app-service/?WT.mc_id=api_CatalogApi'}]
<class 'str'>


现在我们将更新后的消息发送到 LLM，以便我们可以接收自然语言响应，而不是 API JSON 格式的响应。

In [33]:
print("下一次请求中的消息：")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    tools=tools, 
    tool_choice="auto",
    temperature=0
        )  # 从 GPT 获取新的响应，以便它可以看到函数响应


print(second_response.choices[0].message)

下一次请求中的消息：
[{'role': 'user', 'content': 'Find me a good course for a beginner student to learn Azure. Use tools(function-calling)'}, ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_acb35831-f532-4184-8c75-99f9036fed64', function=Function(arguments='{"role":"student","product":"Azure","level":"beginner"}', name='search_courses'), type='function', index=0)]), {'role': 'tool', 'tool_call_id': 'call_0_acb35831-f532-4184-8c75-99f9036fed64', 'content': "[{'title': 'Describe concepts of cryptography', 'url': 'https://learn.microsoft.com/en-us/training/modules/describe-concepts-of-cryptography/?WT.mc_id=api_CatalogApi'}, {'title': 'Introduction to audio classification with TensorFlow', 'url': 'https://learn.microsoft.com/en-us/training/modules/intro-audio-classification-tensorflow/?WT.mc_id=api_CatalogApi'}, {'title': 'Design a Performant Data Model in Azure SQL Database with Azure Data Studi

## 代码挑战 

干得好！要继续学习 OpenAI 函数调用，您可以构建：https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst 
 - 函数的更多参数，这可能有助于学习者找到更多课程。您可以在此处找到可用的 API 参数： 
 - 创建另一个函数调用，该调用从学习者那里获取更多信息，例如他们的母语 
 - 在函数调用和/或 API 调用未返回任何合适课程时创建错误处理